# Installing Necessary Packages

In [87]:
!pip -q install zai-sdk
!pip -q install openai
!pip -q install google-generativeai

# Import Packages

In [88]:
import os
import sys
import re
import time
import json
import random
import logging
import warnings
import requests

import numpy as np
import pandas as pd

from google import genai
from google.genai.types import GenerateContentConfig

from abc import ABC, abstractmethod

from zai import ZaiClient
from openai import OpenAI
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

# Logging

In [89]:
logger = logging.getLogger()      # root logger
logger.setLevel(logging.WARNING)

if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter("%(asctime)s %(levelname)s: %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# System Prompt

In [90]:
PROMPT_TEMPLATE = """
You are an expert clinical NLP annotator extracting structured entities from CHEST CT radiology reports.

OUTPUT: Valid JSON array only. No markdown, no explanation, no comments.

SCHEMA:
[
  {
    "general_finding": "string or None",
    "specific_finding": "string or None",
    "finding_presence": "present | absent | uncertain | None",
    "location": [],
    "degree": [],
    "measurement": "string or None",
    "comparison": "stable | improved | worsened | None"
  }
]

═══════════════════════════════════════════════════
FIELD DEFINITIONS
═══════════════════════════════════════════════════

general_finding: The CLINICAL CATEGORY of what is being described.
  USE the clinical entity name, NOT the anatomical structure, NOT a generic category.

  CORRECT examples:
    "cardiomegaly", "pericardial effusion", "pulmonary embolus",
    "atherosclerotic plaque", "coronary artery calcification",
    "lymphadenopathy", "pleural effusion", "pneumothorax",
    "catheter", "endotracheal tube", "nasogastric tube",
    "hematoma", "fracture", "intramural hematoma",
    "lung parenchyma", "thyroid gland", "heart", "aorta",
    "thoracic aorta", "aortic arch", "pulmonary artery",
    "pulmonary arteries", "trachea", "central airways",
    "airways", "liver", "gallbladder", "bone",
    "lymph nodes", "pulmonary nodules", "pulmonary nodule",
    "soft tissue", "upper abdomen", "spine",
    "cholecystitis", "aortic pathology", "consolidation",
    "vertebral artery", "prior disease", "thyroid nodules",
    "great vessels", "upper abdominal organs"

  WRONG — never use generic categories like:
    ✗ "thyroid abnormality" → use "thyroid gland" or "thyroid nodules"
    ✗ "cardiac abnormality" → use "coronary artery calcification" or "heart"
    ✗ "vascular abnormality" → use "aorta", "pulmonary artery", etc.
    ✗ "parenchymal abnormality" → use "lung parenchyma" or "consolidation"
    ✗ "pleural abnormality" → use "pleural effusion" or "pneumothorax"
    ✗ "skeletal abnormality" → use "bone" or "fracture"
    ✗ "airway abnormality" → use "airways" or "trachea"
    ✗ "lymph node abnormality" → use "lymphadenopathy" or "lymph nodes"
    ✗ "liver lesion" → use "liver"
    ✗ "abdominal abnormality" → use "upper abdomen"

  For devices: general_finding = the device name itself:
    "catheter", "endotracheal tube", "nasogastric tube"

  For absent pathology: general_finding = the pathology name:
    "pericardial effusion" (not "pericardium" or "pericardial space")
    "pulmonary embolus" (not "pulmonary arteries")
    "cholecystitis" (not "gallbladder")
    "lymphadenopathy" (not "lymph nodes")
    "pneumothorax" (not "lungs")

specific_finding: The specific observation or descriptor.
  Use concise clinical terminology, not raw sentence fragments.
  CORRECT: "aberrant origin", "subcutaneous hematoma", "filling defects"
  WRONG: "left vertebral artery arising directly from the aorta" (too verbose)

location: Array of anatomical sites. ALWAYS populate when the report states a location.
  Include laterality. Include both insertion site AND tip position for devices.

degree: Array of qualifiers (mild, moderate, severe, small, focal, diffuse, patchy, etc.).
  Include descriptors like "suspicious", "significant", "bibasilar", "subcentimeter", "scattered".
  For unremarkable/normal findings, degree = ["unremarkable"] or ["normal"].
  Use empty array [] when no qualifiers apply. Never put "None" as a string inside the array.

measurement: Exact numeric value with units from text, or "None".

comparison: Only set if report compares to prior imaging. Otherwise "None".

═══════════════════════════════════════════════════
ENTITY GRANULARITY RULES
═══════════════════════════════════════════════════

RULE 1: ONE SENTENCE DESCRIBING ONE THING = ONE ENTITY
  Each distinct clinical observation gets its own entity.

RULE 2: SPLIT SEPARATE ANATOMICAL STRUCTURES
  "The aorta and great vessels are normal" = TWO entities:
    1. general_finding: "aorta", specific_finding: "aorta course and caliber"
    2. general_finding: "great vessels", specific_finding: "great vessels course and caliber"

  "Trachea and central bronchi are patent" = TWO entities:
    1. general_finding: "trachea", specific_finding: "patency"
    2. general_finding: "bronchi", specific_finding: "patency" (or "central airways"/"patency")

  "The trachea and central airways are patent" = TWO entities:
    1. general_finding: "trachea", specific_finding: "patency"
    2. general_finding: "central airways", specific_finding: "patency"

RULE 3: SPLIT SEPARATE CLINICAL FINDINGS ON SAME STRUCTURE
  "Heart is normal in size with a trace pericardial effusion" = TWO entities:
    1. general_finding: "heart", specific_finding: "heart size", degree: ["normal"]
    2. general_finding: "pericardial effusion", specific_finding: "pericardial effusion", degree: ["trace"]

  "Normal contour and caliber with moderate atherosclerotic plaque" = TWO entities:
    1. Contour/caliber entity
    2. Atherosclerotic plaque entity

  "Thyroid gland is heterogeneous with multiple small nodules in the right lobe" = TWO entities:
    1. general_finding: "thyroid gland", specific_finding: "heterogeneous thyroid", degree: ["heterogeneous"]
    2. general_finding: "thyroid nodules", specific_finding: "nodules", location: ["right thyroid lobe"], degree: ["multiple", "small"]

RULE 4: SEPARATE ABSENT PATHOLOGY FROM RELATED ABSENT PATHOLOGY
  "No surface irregularity to suggest intimal flap, dissection, or atheromatous ulcer"
  = TWO entities:
    1. general_finding: "thoracic aorta", specific_finding: "surface irregularity", finding_presence: "absent"
    2. general_finding: "aortic pathology", specific_finding: "intimal flap, dissection, or atheromatous ulcer", finding_presence: "absent"

RULE 5: MERGE — DEVICES
  Merge device insertion site + tip position into ONE entity with multi-location array.
  "Catheter in right IJ with tip in right atrium" → location: ["right internal jugular", "proximal right atrium"]

RULE 6: MERGE — GROUPED NEGATIVES (same clinical domain)
  "No focal consolidation, effusion or edema" → ONE entity:
    general_finding: "lung parenchyma", specific_finding: "consolidation, effusion, or edema", finding_presence: "absent"
  Include applicable qualifiers: degree: ["focal"]

RULE 7: MERGE — VERSUS / AND/OR STATEMENTS
  "consolidation versus atelectasis" or "atelectasis and/or scarring" → ONE entity:
    finding_presence: "uncertain"
    Keep wording: specific_finding: "consolidation versus atelectasis"

RULE 8: MERGE — COREFERENCES
  "Ground glass opacities are seen. These involve all lobes." → ONE entity

═══════════════════════════════════════════════════
IMPRESSION HANDLING
═══════════════════════════════════════════════════

- If IMPRESSION restates a FINDINGS entity with same information → extract ONLY ONCE (use FINDINGS version)
- If IMPRESSION adds new clinical significance → merge into the existing entity's degree
- Do NOT extract IMPRESSION-only summary statements that are not specific findings
  Example: "NO ACUTE VASCULAR INJURY" → Do NOT extract this as a separate entity (it's a summary)

═══════════════════════════════════════════════════
FINDING PRESENCE
═══════════════════════════════════════════════════

present: "is seen", "demonstrates", "present", "identified", "noted", "appreciated"
absent: "no", "without", "absent", "no evidence of", "negative for"
uncertain: "versus", "and/or", "possible", "suspicious for", "cannot exclude", "may represent"

═══════════════════════════════════════════════════
SPECIAL CASES
═══════════════════════════════════════════════════

NORMAL/UNREMARKABLE: Extract with finding_presence: "present", degree: ["unremarkable"] or ["normal"]
  "thyroid gland is unremarkable" → general_finding: "thyroid gland", degree: ["unremarkable"]
  "upper abdomen is unremarkable" → general_finding: "upper abdomen", degree: ["unremarkable"]

DEVICES: general_finding = device name. Location = [insertion site, tip position]. Measurement = distance if stated.
  "ET tube tip 2.5 cm above carina" → measurement: "2.5 cm above the carina"

SEQUELAE: Extract as separate entity.
  "calcified nodules, likely sequela of prior granulomatous process" → TWO entities:
    1. The calcified nodules
    2. general_finding: "prior disease", specific_finding: "sequela of prior granulomatous process"

LYMPH NODES:
  "no lymphadenopathy" → general_finding: "lymphadenopathy", finding_presence: "absent"
  "subcentimeter nodes present" → general_finding: "lymph nodes", finding_presence: "present", degree: ["subcentimeter"]
  Only extract lymphadenopathy as present if: "enlarged", "prominent", "pathologic"

LIMITED ASSESSMENT:
  "Lack of noncontrast images limits ability to assess for intramural hematoma"
  → general_finding: "intramural hematoma", finding_presence: "uncertain"

═══════════════════════════════════════════════════
COMMON ERRORS TO AVOID
═══════════════════════════════════════════════════

✗ Using generic categories ("vascular abnormality") instead of specific clinical terms
✗ Merging separate anatomical structures into one entity (trachea + bronchi, aorta + great vessels)
✗ Merging separate clinical findings on the same structure (heart size + pericardial effusion)
✗ Missing the heart size entity when it's stated
✗ Missing the pericardial effusion entity when it's stated (even if absent)
✗ Extracting IMPRESSION-only summary lines as new entities
✗ Leaving location empty when the report specifies a location
✗ Putting "None" as a string inside degree array (use empty [] instead)
✗ Missing qualifiers like "significant", "scattered", "bibasilar" in degree
✗ Using raw sentence fragments as specific_finding instead of concise terms

═══════════════════════════════════════════════════

NOW EXTRACT FROM THIS REPORT:

<<<REPORT_TEXT>>>

Output ONLY the JSON array.



"""

# API Keys

In [91]:
user_secrets = UserSecretsClient()

API_KEYS = {
    "gemini": user_secrets.get_secret("gemini_api_key_0"),
    "gemma": user_secrets.get_secret("gemini_api_key_0"),
    #"glm": user_secrets.get_secret("glm_api_key"),
    #"deepseek": user_secrets.get_secret("deepseek_api_key"),
}

# LLM Classes

In [92]:
class AIBaseModel(ABC):
    def __init__(self, api_key: str, model_name: str):
        self.api_key = api_key
        self.model_name = model_name
    
    @abstractmethod
    def invoke(self, prompt: str, **kwargs):
        raise NotImplementedError

In [93]:
class GeminiModel(AIBaseModel):
    def __init__(self, api_key: str, model_name: str = "gemini-2.5-flash"):
        self.model_name = model_name
        self.client = genai.Client(api_key=api_key)
        self.sleep_time = self._get_time_to_sleep()
        
    def _get_time_to_sleep(self):
        requests_per_minute = 15  # default
        
        if self.model_name == "gemini-2.5-flash":
            requests_per_minute = 5
        elif self.model_name == "gemini-3-flash-preview":
            requests_per_minute = 5
        elif self.model_name == "gemini-2.5-flash-lite":
            requests_per_minute = 10
        elif self.model_name == "gemini-1.5-flash":
            requests_per_minute = 15
        elif "gemma" in self.model_name:
            requests_per_minute = 30
            
        return 60 / requests_per_minute
    
    def invoke(
        self, 
        prompt: str,
        system_prompt: str | None = None,
        temperature: float = 0, 
        top_p: float = 1, 
        max_tokens: int = 8192,
    ):
        try:
            response = self.client.models.generate_content(
                model=self.model_name,
                contents=prompt,
                config=GenerateContentConfig(
                    system_instruction=None,
                    temperature=temperature,
                    top_p=top_p,
                    max_output_tokens=max_tokens,
                ),
            )

            if hasattr(response, "candidates"):
                texts = []
                for c in response.candidates:
                    for p in getattr(c.content, "parts", []):
                        if getattr(p, "text", None):
                            texts.append(p.text)
                return "\n".join(texts) if texts else None
            return None
        except Exception as e:
            logger.error(f"Gemini API error: {e}")
        return None

In [94]:
class GLMModel(AIBaseModel):
    def __init__(self, api_key: str, model_name: str = "glm-4.5-flash"):
        super().__init__(api_key, model_name)
        self.client = ZaiClient(api_key=api_key)
        self.sleep_time = 6
        
    def invoke(
        self,
        prompt: str,
        system_prompt: str | None = None,
        temperature: float = 0.0,
        top_p: float = 1.0,
        max_tokens: int = 8192,
    ):
        try:
            messages = [
                {
                    "role": "system", 
                    "content": "You are a medical NLP system specialized in medical entity extraction from a given radiology report."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens,
                stream=False,
            )

            if response.choices:
                return response.choices[0].message.content.strip()

            return None
        except Exception as e:
            logger.error(f"GLM API error: {e}")
            if hasattr(e, "status_code"):
                logger.error(f"Status code: {e.status_code}")
            if hasattr(e, "body"):
                logger.error(f"Error body: {e.body}")
            return None

In [95]:
class DeepSeekModel(AIBaseModel):
    def __init__(self, api_key: str, model_name: str = "deepseek-chat"):
        super().__init__(api_key, model_name)
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://api.deepseek.com"
        )
        self.sleep_time = 3
        
    def invoke(
        self,
        prompt: str,
        system_prompt: str | None = None,
        temperature: float = 0.0,
        top_p: float = 1.0,
        max_tokens: int = 8192,
    ):
        try:
            messages = [
                {
                    "role": "system", 
                    "content": "You are a medical NLP system specialized in medical entity extraction from a given radiology report."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]

            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens,
                stream=False,
            )

            return response.choices[0].message.content.strip()
        except Exception as e:
            logger.error(f"DeepSeek API error: {e}")
            return None


## Get Model

In [96]:
def get_ai_model(model: str, model_name: str):
    model_map = {
        "gemini": GeminiModel,
        "glm": GLMModel,
        "deepseek": DeepSeekModel
    }
    
    if model not in model_map:
        raise ValueError(f"Unvalid model: {model}. Choices: {list(model_map.keys())}")
    
    return model_map[model](API_KEYS[model], model_name)

In [97]:
def load_jsonl(path: str):
    with open(path, encoding="utf-8") as file:
        return [json.loads(line) for line in file if line.strip()]

def load_json(path: str):
    with open(path, encoding="utf-8") as file:
        return json.load(file)

In [98]:
def build_prompt(report: str) -> str:
    return PROMPT_TEMPLATE.replace("<<<REPORT_TEXT>>>", report)


In [99]:
def safe_parse_json(text: str):
    if not text:
        return None

    text = re.sub(r"```json|```", "", text, flags=re.IGNORECASE).strip()

    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass

    # önce array yakala
    m = re.search(r"\[.*\]", text, re.S)
    if m:
        try:
            return json.loads(m.group())
        except Exception:
            pass

    # sonra object
    m = re.search(r"\{.*\}", text, re.S)
    if m:
        try:
            return json.loads(m.group())
        except Exception:
            return None

    return None


In [100]:
def run_inference_radgraph(
    dataset,
    output_path: str,
    model_id: str,
    model_name: str,
):
    model = get_ai_model(model_id, model_name)

    results = []
    for idx, sample in tqdm(enumerate(dataset), total=len(dataset), desc="Processing Samples"):
        
        prompt = build_prompt(sample["report"])

        raw_output = model.invoke(
            prompt=prompt,
            system_prompt=None
        )

        parsed = safe_parse_json(raw_output)

        print(f"Report: {sample['report']}")
        print(f"Parsed Output: {parsed}")
        
        result = {
            "dataset": sample["dataset"],
            "doc_key": sample["doc_key"],
            "report": sample["report"],
            "model": model_name,
            "entities": parsed,
        }
        
        results.append(result)

        if (idx + 1) % 5 == 0:
            temp_path = output_path.replace(".json", "_temp.json")
            with open(temp_path, "w") as f:
                f.write(json.dumps(results))
        

        time.sleep(model.sleep_time + 0.2)

    with open(output_path, "w") as f:
        for r in results:
            f.write(json.dumps(r) + "\n")

    return results

def run_inference_ratener(
    dataset,
    output_path: str,
    model_id: str,
    model_name: str,
):
    model = get_ai_model(model_id, model_name)

    results = []
    for idx, sample in tqdm(enumerate(dataset), total=len(dataset), desc="Processing Samples"):
        
        prompt = build_prompt(sample["report"])

        raw_output = model.invoke(
            prompt=prompt,
            system_prompt=None
        )

        parsed = safe_parse_json(raw_output)

        print(f"Report: {sample['report']}")
        print(f"Raw Output: {raw_output}")
        
        result = {
            "note_id": sample["note_id"],
            "report": sample["report"],
            "model": model_name,
            "entities": parsed,
        }
        
        results.append(result)

        if (idx + 1) % 5 == 0:
            temp_path = output_path.replace(".json", "_temp.json")
            with open(temp_path, "w") as f:
                f.write(json.dumps(results))
        

        time.sleep(model.sleep_time + 0.2)

    with open(output_path, "w") as f:
        for r in results:
            f.write(json.dumps(r) + "\n")

    return results

In [101]:
INPUT_PATH = "/kaggle/input/random20/radgraphxl-chest-ct-sample-20.json"
OUTPUT_PATH = "/kaggle/working/chest-ct-schema-random20.json"

dataset = load_json(INPUT_PATH)

results = run_inference_radgraph(
    dataset=dataset,
    output_path=OUTPUT_PATH,
    model_id="gemini",
    model_name="gemma-3-27b-it",
)

Processing Samples:   0%|          | 0/18 [00:00<?, ?it/s]

Report: CT Pulmonary Embolism Chest 12 - 05 - 06 HISTORY : 73 - year - old male with history of renal cell carcinoma now with chest pain and shortness of breath . Possibly pneumonia in the left upper lobe . Evaluate for pulmonary embolism . COMPARISON : CT chest abdomen and pelvis 10 - 19 - 2006 . TECHNIQUE : Multiple contiguous axial images were obtained from the thoracic inlet to the domes of the diaphragms utilizing the standard CT pulmonary embolism chest protocol . 100 mL of Isovue - 300 was given without adverse reaction . Additional 3 - D reconstructed images were performed and reviewed . FINDINGS : Subcentimeter hypodense lesions are seen in the thyroid gland . A large subcarinal lymph node is now noted measuring 21 mm which is new since the prior study . Left hilar / AP window lymphadenopathy has increased in size from prior study , now measuring 3 . 1 x 5 . 1 cm ( previously 2 . 8 x 3 . 8 cm ) . The heart is normal in size . There is no pericardial effusion . Coronary arteria

Processing Samples:   6%|▌         | 1/18 [00:32<09:08, 32.25s/it]

Report: Addendum Begins Coronal and sagittal MPR and MIP reformations were performed on an off - line 3D workstation and corroborate the findings . Addendum Ends CT PULMONARY ANGIOGRAM : 1 / 20 / 2021 COMPARISON : None . CLINICAL HISTORY : Acute on chronic dyspnea . TECHNIQUE : After administration of IV contrast , a pulmonary CT angiogram was obtained . FINDINGS : VASCULAR : Evaluation is limited at the bases of the lungs by motion . No evidence of pulmonary embolism . The pulmonary artery measures 27 mm in diameter , which is within normal limits . There is a tight stenosis of the origin of the left carotid artery and a mild stenosis of the left subclavian artery , just distal to its origin . Diffuse aortic calcifications are seen . In addition , there is a small splenic artery aneurysm at the splenic hilum measuring 10 mm in diameter . The aneurysm is heavily calcified peripherally . CHEST AND MEDIASTINUM : There is severe centrilobular emphysema , trending towards being panlobular 

Processing Samples:  11%|█         | 2/18 [01:01<08:04, 30.26s/it]

...

Processing Samples:  89%|████████▉ | 16/18 [09:23<01:21, 40.79s/it]

Report: EXAM : CT Pulmonary Embolism Chest Only DATE : 8 - 4 - 2019 5 : 32 PM CLINICAL HISTORY : Hypoxia , dyspnea on exertion in a patient with chest pain and hypotension . COMPARISON : Chest x - ray from earlier same day august 04th CONTRAST : 115 cc of Isovue - 300 TECHNIQUE : Following uneventful administration of iodinated intravenous contrast media , multidetector helical CT data acquisition was performed through the thorax during pulmonary arterial phase of contrast . Multiplanar reformatted , 3 - D volume rendered , and maximum intensity projection images were reviewed on an independent workstation . Based on a 32 cm body phantom , the estimated radiation dose ( CTDIvol mGy ) for each series in this exam are : 22 . 5 , 28 . 9 mGy . The estimated cumulative dose ( DLP mGy - cm ) is : 853 . 9 mGy - cm . NOTE : The radiation dose indicators for CT -- the ' volume CT Dose Index ' ( CTDIvol ) given in milli - Gray ( mGy ), and the Dose Length Product ( DLP ) given in mGy - centimete

Processing Samples:  94%|█████████▍| 17/18 [09:53<00:37, 37.48s/it]

Report: EXAM : CT Angio Thorax , 1 - 12 - 17 9 : 24 AM HISTORY : 25 years Male presents to the emergency department after a motor vehicle collision . COMPARISON : Chest radiograph 1 - 12 - 17 TECHNIQUE : After the uneventful administration of intravenous contrast , multiple contiguous axial CT images were acquired through the chest during the arterial phase after contrast administration . Coronal and sagittal reformatted images , as well as maximum intensity images were provided . CONTRAST : Omnipaque 350 , 120 mL . DOSE : Based on a 32 cm body phantom , the estimated radiation dose ( CTDIvol mGy ) for each series in this exam are : 32 . 83 , 16 . 83 , 18 . 19 , 8 . 93 mGy . The estimated cumulative dose ( DLP mGy - cm ) is : 1806 mGy - cm . NOTE : The radiation dose indicators for CT -- the ' volume CT Dose Index ' ( CTDIvol ) given in milli - Gray ( mGy ), and the Dose Length Product ( DLP ) given in mGy - centimeters ( mGy . cm ) -- are generated from the CT scanner to estimate radi

Processing Samples: 100%|██████████| 18/18 [10:26<00:00, 34.81s/it]


## Saving as Pretty JSON 

In [102]:
import json

INPUT_PATH = "/kaggle/working/chest-ct-schema-random20.json"
OUTPUT_PATH = "/kaggle/working/chest-ct-schema.pretty-random20.jsonl"

with open(INPUT_PATH, "r", encoding="utf-8") as fin, open(OUTPUT_PATH, "w", encoding="utf-8") as fout:
    for line in fin:
        line = line.strip()
        if not line:
            continue

        obj = json.loads(line)

        # her kaydı indent'li yaz
        fout.write(json.dumps(obj, ensure_ascii=False, indent=2))
        fout.write("\n\n")  # kayıtlar arası boşluk

print("Saved ->", OUTPUT_PATH)


Saved -> /kaggle/working/chest-ct-schema.pretty-random20.jsonl


In [103]:
# INPUT_PATH = "/kaggle/input/radgraph/stanford-radgraph-XL-sentence.jsonl"
# OUTPUT_PATH = "/kaggle/working/stanford-radgraph-XL-mapped.jsonl"

# dataset = load_jsonl(INPUT_PATH)[1:]

# results = run_inference_radgraph(
#     dataset=dataset,
#     output_path=OUTPUT_PATH,
#     model_id="deepseek",
#     model_name="deepseek-chat",
# )